In [ ]:
!pip install -q transformers datasets peft accelerate torch


In [ ]:
!nvidia-smi


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model


In [ ]:

data = [
    {
        "text": "Instruction: Say hello\nResponse: Hello!"
    },
    {
        "text": "Instruction: What is AI?\nResponse: AI stands for Artificial Intelligence."
    },
    {
        "text": "Instruction: Capital of France?\nResponse: Paris."
    }
]

dataset = Dataset.from_list(data)


In [ ]:
model_name = "gpt2-medium"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized_ds = dataset.map(tokenize, batched=True)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


In [ ]:
training_args = TrainingArguments(
    output_dir="./lora_out",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    fp16=True,
    logging_steps=1,
    report_to="none",
    save_strategy="no"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer
)


In [ ]:
trainer.train()
